In [1]:
#Codigo para clasificar mensajes recibidos + matchear (SOLO CONFIRMACION DE PROPERATI)
#Ultima actualizacion: 6/05/2022

#Inputs: (A) base de scraps de properati (randomization) y (B) base de mails con respuestas, en las cuales haya confirmaciones de properati
#Outputs: (A) sent_mails_conf_envio_[FECHA], que agrupa las publicaciones a las que les enviamos correctamente el mail, y (B) not_sent_mails_conf_envio_[FECHA]_[PAIS] que muestra, para cada país, las inmobiliarias a las que les tenemos que reenviar los correos.

In [2]:
# Importar paquetes necesarios para la clasificacion

import pandas as pd
import numpy as np
from datetime import datetime
import matching_v1 as matching

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

pd.options.mode.chained_assignment = None  # default='warn'

#### Ronda y fecha de la ronda
ronda = '1'
fecha = '2022_04_25'   # FIXME: cambiar fecha!! Está mal!!
fecha_rand = '2022_04_19'

#### Establecer rutas de entrada y salida para las bases (si se cambian, comentar las que no se usen)
#Rutas Nico
path_dropbox = r"C:\Users\Nico\Dropbox"

#Rutas Juli
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#Rutas Luis
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#### Rutas utilizadas:
path_data = path_dropbox + r"\BID-LGBTQ+\campo\data"
path_dataout = path_data + r"\mails_matches\matched\confirmaciones_de_envio"
path_datain = path_data + r"\mails_matches\to_match"
path_deleted = path_data + r"\mails_matches\deleted"

In [3]:
#Importar bases de scraps (randomization) de todos los paises
scrap_arg = pd.read_stata(path_data + fr'\randomization\round{ronda}_{fecha_rand}\randomization_ARG_{fecha_rand}_round_{ronda}.dta')
scrap_col = pd.read_stata(path_data + fr'\randomization\round{ronda}_{fecha_rand}\randomization_COL_{fecha_rand}_round_{ronda}.dta')
scrap_ecu = pd.read_stata(path_data + fr'\randomization\round{ronda}_{fecha_rand}\randomization_ECU_{fecha_rand}_round_{ronda}.dta')
scrap_per = pd.read_stata(path_data + fr'\randomization\round{ronda}_{fecha_rand}\randomization_PER_{fecha_rand}_round_{ronda}.dta')
scrap_full_no_backup = pd.concat([scrap_arg,scrap_col,scrap_ecu,scrap_per])

#Importar bases de scraps (randomization backups) de la ronda
scrap_arg_backup = pd.read_stata(path_data + fr'\randomization\backup\round{ronda}\randomization_ARG_{fecha_rand}_round_{ronda}_backup1.dta')
scrap_arg_backup2 = pd.read_stata(path_data + fr'\randomization\backup\round{ronda}\randomization_ARG_2022_04_19_round_1_v2scrape.dta')
scrap_col_backup = pd.read_stata(path_data + fr'\randomization\backup\round{ronda}\randomization_COL_{fecha_rand}_round_{ronda}_backup1.dta')
scrap_ecu_backup = pd.read_stata(path_data + fr'\randomization\backup\round{ronda}\randomization_ECU_{fecha_rand}_round_{ronda}_backup1.dta')
scrap_per_backup = pd.read_stata(path_data + fr'\randomization\backup\round{ronda}\randomization_PER_{fecha_rand}_round_{ronda}_backup1.dta')
scrap_full_backup = pd.concat([scrap_arg,scrap_col,scrap_ecu,scrap_per])

###Construir una unica base uniendo las de los cuatro paises
scrap_full = pd.concat([scrap_arg,scrap_col,scrap_ecu,scrap_per,
                        scrap_arg_backup,scrap_arg_backup2,scrap_col_backup,scrap_ecu_backup,scrap_per_backup])
scrap_full = scrap_full.reset_index()
scrap_full.drop(columns='index')

scrap_full['script_string_3'] = scrap_full['script_string_3'].astype('str')
scrap_full.drop(columns=['index','Unnamed01','Unnamed011'], inplace=True)

In [4]:
print(scrap_full_backup.duplicated(subset=['inmobiliaria','municipio']).value_counts())
print(scrap_full_no_backup.duplicated(subset=['inmobiliaria','municipio']).value_counts())

# Nota: las que se repiten son unicamente entre backups y no backups... Son las que fallaron y tuvimos que enviar de nuevo!

False    3696
dtype: int64
False    3696
dtype: int64


In [5]:
#Importar base de mails recibidos 
rtas_full = pd.read_excel(path_datain + fr"\round_{ronda}\mails_round_{ronda}_{fecha}.xlsx") # Ultima descarga de gmail

In [6]:
# Base de correos desde los que enviamos
mails_df = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\0_instrucciones\2-Emails para Intervención.xlsx")
mails_df['Usuario_clean'] = mails_df.Usuario.apply(matching.clean_mail)
users = mails_df['Usuario_clean'].to_list()

In [7]:
#### Elimino observaciones que se corresponden a mails de prueba o de Google

# Dropeo observaciones que tienen Google en el mail / user
prev = len(rtas_full)
rtas_full_out = rtas_full[rtas_full['from'].str.contains("Google", na=False) == True]
rtas_full =     rtas_full[rtas_full['from'].str.contains("Google", na=False) == False]
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba de Juli
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]
rtas_full = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]])

print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == True]
rtas_full = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba (from == to, ¡nos los autoenviamos!)
prev = len(rtas_full) 
for user in users:
    to_concat = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==True ]
    rtas_full = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==False]
    rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Exporto las obs dropeadas
# rtas_full_out.to_excel(path_deleted + fr"\observaciones_eliminadas_round_{ronda}_{fecha}.xlsx")

Se dropearon 208 observaciones.
Se dropearon 226 observaciones.
Se dropearon 5 observaciones.
Se dropearon 8 observaciones.


In [8]:
##### Identifica mails que vienen de properati 'contacts@mailing.properati.com'

# Filtro nulls
before = len(rtas_full)
rtas_full = rtas_full.dropna(subset=['from','body'])
after = len(rtas_full)
print("Se dropearon", before-after, "observaciones.")

# Creo binaria con True para los mails de properati
mails_properati = ['contacts@mailing.properati.com']

rtas_full['conf_envio'] = rtas_full['from'].apply(matching.match_string_con_lista, lista_matchs=mails_properati)
print("Match confirmaciones realizado.\nCantidad de mails de @properati:")

# Reporta los resultados
rtas_full['conf_envio'].value_counts()

Se dropearon 12 observaciones.
Match confirmaciones realizado.
Cantidad de mails de @properati:


True     3645
False    1980
Name: conf_envio, dtype: int64

In [9]:
##### Limpio las fechas
rtas_full['date_clean'] = rtas_full['date'].str.replace(".+,\s","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("\-|\+|(0\d00)(.*)","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("GMT","")
rtas_full['date_clean'] = rtas_full['date_clean'].str.lstrip(' ').str.rstrip(' ')
rtas_full['date_clean'] = rtas_full['date_clean'].str.zfill(20)
rtas_full['date_clean'] = pd.to_datetime(rtas_full['date_clean'], format="%d %b %Y %H:%M:%S")
rtas_full['date_clean'] = rtas_full['date_clean'].dt.date  # Solo me quedo dia-mes-año

In [13]:
import importlib
importlib.reload(matching)

##### Matcheo @properati con scraps
fin_ronda_1 = pd.to_datetime("2022-05-03").date()
rtas_properati = rtas_full.loc[(rtas_full['date_clean']<=fin_ronda_1) & (rtas_full['conf_envio']==True)]

today = datetime.now()
today = today.strftime("%Y_%m_%d")
nombre = f'mails_conf_envio_round_{ronda}_{today}'

rtas_properati['index_scrap'] = np.nan
rtas_properati['match_confiab'] = np.nan
rtas_properati['match_step'] = np.nan
rtas_properati['match_string'] = np.nan
rtas_properati['match2_string'] = np.nan
scrap_full['index_rtas'] = np.nan
scrap_full['match_confiab'] = np.nan
scrap_full['match_step'] = np.nan

matched_arg = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='arg'], scrap_full=scrap_full[scrap_full['pais']=='arg'], pais = '_ARG')
matched_col = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='col'], scrap_full=scrap_full[scrap_full['pais']=='col'], pais = '_COL')
matched_ecu = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='ecu'], scrap_full=scrap_full[scrap_full['pais']=='ecu'], pais = '_ECU')
matched_per = matching.full_proceso_solo_id(path=path_dataout, rtas_full=rtas_properati[rtas_properati['pais']=='per'], scrap_full=scrap_full[scrap_full['pais']=='per'], pais = '_PER')

# Exporto los matcheados a un único Excel
matched = pd.concat([matched_arg,matched_col,matched_ecu,matched_per])
matched.to_excel(path_dataout + "\\sent_" + nombre +".xlsx")
print(path_dataout + "\\sent_" + nombre +".xlsx creado")

['rodriguezmanuel.w@gmail.com' 'rodriguezmanuelw@gmail.com'
 'gomez.pablo.u@gmail.com' 'gomez.pablou@gmail.com'
 'rodriguez.manuelw@gmail.com' 'gomezpablo.u@gmail.com'
 'rodriguez.manuel.w@gmail.com' 'gomezpablou@gmail.com'
 'jfmanuelrodriguez@gmail.com' 'jfmanuel.rodriguez@gmail.com'
 'phpablo.gomez@gmail.com' 'jf.manuelrodriguez@gmail.com'
 'ph.pablo.gomez@gmail.com' 'ph.pablogomez@gmail.com'
 'jf.manuel.rodriguez@gmail.com' 'phpablogomez@gmail.com'
 'manuelrodriguez.jf@gmail.com' 'manuel.rodriguez.jf@gmail.com'
 'manuelrodriguezjf@gmail.com' 'pablomngomez@gmail.com'
 'pablo.mngomez@gmail.com' 'pablomn.gomez@gmail.com'
 'pablo.mn.gomez@gmail.com' 'manuel.rodriguezjf@gmail.com'
 'fh.pablo.gomez@gmail.com' 'fhpablogomez@gmail.com'
 'jc.manuel.rodriguez@gmail.com' 'jcmanuelrodriguez@gmail.com'
 'jc.manuelrodriguez@gmail.com' 'fh.pablogomez@gmail.com'
 'fhpablo.gomez@gmail.com' 'jcmanuel.rodriguez@gmail.com'
 'rodriguez.jc.manuel@gmail.com' 'pablogomezfh@gmail.com'
 'rodriguezjc.manuel@g

In [28]:
matched._merge.value_counts()

#### Nota: ##########################################
# Los 9 right_only son todos de la propiedad de Ine!!
#####################################################

matched = matched[-matched.body.str.contains("miberniel")]

both          3633
left_only        0
right_only       0
Name: _merge, dtype: int64

In [29]:
matched[matched.id]

,inmobiliaria,fecha,link,municipio,ciudad,pais_x,descripcion,precio,moneda,caracteristicas,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,id,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,m2_above_mdn,common_name,capital,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,script_string_3,randomization_seed,nombre,nombre_2,profesional,vivienda,n_adds_muni,n_adds_muni_above_mdn,random,index_rtas,match_confiab_x,match_step_x,Unnamed: 0,subject,from,to,body,date,cc,thread_id,mimeType,pais_y,conf_envio,date_clean,index_scrap,match_confiab_y,match_step_y,match_string,match2_string,_merge


In [32]:
#### TODO:
# Verificar manualmente que estos estén bien. El id de properati es muy corto así que accidentalmente podría generar errores de matcheo.
matched[matched.id.str.len()<5]

,inmobiliaria,fecha,link,municipio,ciudad,pais_x,descripcion,precio,moneda,caracteristicas,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,id,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,m2_above_mdn,common_name,capital,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,script_string_3,randomization_seed,nombre,nombre_2,profesional,vivienda,n_adds_muni,n_adds_muni_above_mdn,random,index_rtas,match_confiab_x,match_step_x,Unnamed: 0,subject,from,to,body,date,cc,thread_id,mimeType,pais_y,conf_envio,date_clean,index_scrap,match_confiab_y,match_step_y,match_string,match2_string,_merge
6,heredia,2022-02-18,properati.com.ar/detalle/4ae45_alquiler_depart...,caba,buenos aires ampliado,arg,"edificio de 14 pisos, 5 departamentos por piso...",32000.0,$,,NaN,NaN,NaN,NaN,NaN,,150,32000.0,NaN,NaN,5.0,1.0,NaN,2.0,1.0,hetero,high,Pablo,gomez.pablo.u@gmail.com,Juan,María,1.0,"Buenos días, mi nombre es Pablo, junto a mi pa...",Pablo y María.,Código 150,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,100.0,1.0,92,Hiciste una consulta por la propiedad Neuquen 602,Properati <contacts@mailing.properati.com.ar>,gomez.pablo.u@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Tue, 19 Apr 2022 16:59:01 +0000 (UTC)",NaN,18042c36649755f4,multipart/alternative,arg,True,2022-04-19,21.0,100.0,1.0,"[Match(start=1410, end=1419, dist=0, matched='...",nan,both
49,raices puerto madero,2021-09-12,properati.com.ar/detalle/3z6bc_alquiler_depart...,caba,buenos aires ampliado,arg,se trata de un departamento en el edificio kav...,900.0,usd,,NaN,NaN,NaN,NaN,NaN,,1373,174150.0,NaN,NaN,1.0,0.0,NaN,1.0,1.0,hetero,neutral,Manuel,jf.manuel.rodriguez@gmail.com,Luis,Laura,1.0,"Buenos días, mi nombre es Manuel, junto a mi p...",Manuel y Laura.,Código 1373,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,100.0,1.0,2243,Hiciste una consulta por la propiedad Florida ...,Properati <contacts@mailing.properati.com.ar>,jf.manuel.rodriguez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Tue, 19 Apr 2022 16:31:36 +0000 (UTC)",NaN,18042aa4a41e721a,multipart/alternative,arg,True,2022-04-19,23.0,100.0,1.0,"[Match(start=1365, end=1375, dist=0, matched='...",nan,both
17,inmobiliaria sergio andre,2022-04-18,properati.com.ar/detalle/4foc6_alquiler_depart...,jose_c_paz,buenos aires ampliado,arg,departamento dos ambientes con lavadero y coch...,35000.0,$,"['2ambientes', '1bano', '40 m²totales']",2.0,NaN,1.0,40.0,NaN,,6070,35000.0,875.000000,0.0,8.0,1.0,0.0,1.0,0.0,homo,neutral,Manuel,jcmanuel.rodriguez@gmail.com,Luis,,2.0,"Hola, somos una pareja interesada en alquilar ...",Manuel y Luis,Código 6070,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,100.0,1.0,2639,Hiciste una consulta por la propiedad Arenales...,Properati <contacts@mailing.properati.com.ar>,jcmanuel.rodriguez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 20 Apr 2022 14:24:31 +0000 (UTC)",NaN,180475c4af0edc24,multipart/alternative,arg,True,2022-04-20,26.0,100.0,1.0,"[Match(start=1255, end=1265, dist=0, matched='...",nan,both
15,heredia,2022-02-18,properati.com.ar/detalle/4adue_alquiler_depart...,vicente_lopez,buenos aires ampliado,arg,"edificio de 9 pisos, 4 departamentos por piso,...",40000.0,$,,NaN,NaN,NaN,NaN,NaN,,194,40000.0,NaN,NaN,5.0,1.0,NaN,2.0,0.0,trans,neutral,Pablo,gomez.fh.pablo@gmail.com,Luis,María,1.0,"Buenos días, mi nombre es Pablo, junto a mi pa...",Pablo y María.,Código 194,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,100.0,1.0,1469,Hiciste una consulta por la propiedad Sanchez ...,Properati <contacts@mailing.properati.com.ar>,gomez.fh.pablo@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 20 Apr 2022 17:52:19 +0000 (UTC)",NaN,180481a8a38daedc,multipart/alternative,arg,True,2022-04-20,29.0,100.0,1.0,"[Match(start=1273, end=1282, dist=0, matched='...",nan,both
5,administracion capogrosso,2022-04-06,properati.com.ar/detalle/4eukm_alquiler_depart...,rosario,rosario ampliado,arg,descripcion: es un departamento semipiso. con ...,380